In [1]:
import os
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage
from azure.servicebus.management import ServiceBusAdministrationClient
CONNECTION_STR = 'Endpoint=sb://sharkfinmq.servicebus.windows.net/;SharedAccessKeyName=brokerDev;SharedAccessKey=ethbk8ZhqarR89v455j9psqsBPPa/CcjdtxZY7+3vVw='


In [2]:
def create_queue(servicebus_mgmt_client, qname):
    print("-- Create Queue")
    #qname = qname + "_" + QUEUE_ID
    servicebus_mgmt_client.create_queue(qname, max_delivery_count=10, dead_lettering_on_message_expiration=True)
    print("Queue {} is created.".format(qname))
    print("")


def delete_queue(servicebus_mgmt_client, qname):
    print("-- Delete Queue")
    servicebus_mgmt_client.delete_queue(qname)
    print("Queue {} is deleted.".format(qname))
    print("")


def list_queues(servicebus_mgmt_client):
    print("-- List Queues")
    for queue_properties in servicebus_mgmt_client.list_queues():
        print("Queue Name:", queue_properties.name)
    print("")


def get_and_update_queue(servicebus_mgmt_client, qname):
    print("-- Get and Update Queue")
    queue_properties = servicebus_mgmt_client.get_queue(qname)
    print("Queue Name:", queue_properties.name)
    print("Queue Settings:")
    print("Auto Delete on Idle:", queue_properties.auto_delete_on_idle)
    print("Default Message Time to Live:", queue_properties.default_message_time_to_live)
    print("Dead Lettering on Message Expiration:", queue_properties.dead_lettering_on_message_expiration)
    print("Please refer to QueueDescription for complete available settings.")
    print("")
    # update by updating the properties in the model
    queue_properties.max_delivery_count = 5
    servicebus_mgmt_client.update_queue(queue_properties)

    # update by passing keyword arguments
    queue_properties = servicebus_mgmt_client.get_queue(qname)
    servicebus_mgmt_client.update_queue(queue_properties, max_delivery_count=3)


def get_queue_runtime_properties(servicebus_mgmt_client, qname):
    print("-- Get Queue Runtime Properties")
    queue_runtime_properties = servicebus_mgmt_client.get_queue_runtime_properties(qname)
    print("Queue Name:", queue_runtime_properties.name)
    print("Queue Runtime Properties:")
    print("Updated at:", queue_runtime_properties.updated_at_utc)
    print("Size in Bytes:", queue_runtime_properties.size_in_bytes)
    print("Message Count:", queue_runtime_properties.total_message_count)
    print("Active Message Count:", queue_runtime_properties.active_message_count)
    print("Scheduled Message Count:", queue_runtime_properties.scheduled_message_count)
    print("Dead-letter Message Count:", queue_runtime_properties.dead_letter_message_count)
    print("Please refer to QueueRuntimeProperties from complete available runtime properties.")
    print("")


def init_az_rpc(self,CONNECTION_STR):
    self.servicebus_mgmt_client = ServiceBusAdministrationClient.from_connection_string(CONNECTION_STR)
    self.service_bus_message_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR, logging_enable=True)
    
def create_queue(servicebus_mgmt_client, qname):
    print("-- Create Queue")
    #qname = qname + "_" + QUEUE_ID
    servicebus_mgmt_client.create_queue(qname, max_delivery_count=10, dead_lettering_on_message_expiration=True)
    print("Queue {} is created.".format(qname))
    print("")
    
def send_simple_message(service_bus_client, qname, message):
    sender = service_bus_client.get_queue_sender(queue_name=qname)
    sender.send_messages(message)
    print("Sent a single message")
    
def send_rpc_message(service_bus_client, qname, rpc_message):
    sender = service_bus_client.get_queue_sender(queue_name=qname)
    sender.send_messages(message)
    print f"(Sent RPC awaiting reply from queue: {qname}...)"

def new_rpc_message(msg_body, session_id, replyto, corolationID, lable):
    message = ServiceBusMessage(
    msg_body,
    session_id = session_id,
    replyto = replyto,
    corolationID = corolationID
    application_properties = {'placeholdermetadata': 'custom_data_example'},
    time_to_live = datetime.timedelta(seconds=30),
    label=label
    return message
)
    

In [ ]:
message = ServiceBusMessage(
    "Hello World!!",
    session_id="MySessionID",
    application_properties={'data': 'custom_data'},
    time_to_live=datetime.timedelta(seconds=30),
    label='MyLabel'
)

 Simple request/reply:
        A publisher sends a message into a queue and expects a reply from the message consumer.
        To receive the reply, the publisher owns a queue into which it expects replies to be delivered. 
        The address of that queue is expressed in the ReplyTo property of the outbound message. 
        When the consumer responds, it copies the MessageId of the handled message into 
        the CorrelationId property of the reply message and delivers the message to the destination
        indicated by the ReplyTo property. One message can yield multiple replies, 
        depending on the application context.
    

In [3]:
#sending function


def send_single_message(service_bus_client, qname, message):
    sender = service_bus_client.get_queue_sender(queue_name=qname)
    sender.send_messages(message)
    print("Sent a single message")



In [6]:
service_bus_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR, logging_enable=True)
qname = "chum1903q"
messtxt = (f"This is the 8th chum message on Azure! Sent to queue {qname}")
message = ServiceBusMessage(messtxt)
send_single_message(service_bus_client, qname, message)


Sent a single message


In [7]:
#receiver functions

qname = "chum1903q"

receiver = service_bus_client.get_queue_receiver(queue_name=qname, max_wait_time=5)
for msg in receiver:
    print("Received: " + str(msg))
    receiver.complete_message(msg)

Received: This is the sixth chum message on Azure! Sent to queue chum1903q
Received: This is the 7th chum message on Azure! Sent to queue chum1903q
Received: This is the 8th chum message on Azure! Sent to queue chum1903q


ConnectionClose("ErrorCodes.ConnectionCloseForced: The connection was inactive for more than the allowed 300000 milliseconds and is closed by container 'LinkTracker'. TrackingId:4f748a1b683f42afa2f9e5e50a9933d0_G4, SystemTracker:gateway7, Timestamp:2022-06-16T14:00:07")
ConnectionClose("ErrorCodes.ConnectionCloseForced: The connection was inactive for more than the allowed 300000 milliseconds and is closed by container 'LinkTracker'. TrackingId:a292b193a4eb49b48d508211a2f50146_G39, SystemTracker:gateway7, Timestamp:2022-06-16T14:01:12")
ConnectionClose("ErrorCodes.ConnectionCloseForced: The connection was inactive for more than the allowed 300000 milliseconds and is closed by container 'LinkTracker'. TrackingId:a98b5d99215244d0a048c51e26ff9702_G17, SystemTracker:gateway7, Timestamp:2022-06-16T14:01:16")
ConnectionClose('ErrorCodes.UnknownError: Connection in an unexpected error state.')


In [ ]:
sharkfinMQAdminclient = ServiceBusAdministrationClient.from_connection_string(CONNECTION_STR)
containerID = "chum1903q"
create_queue(sharkfinMQAdminclient, containerID)


In [ ]:
#get_and_update_queue(sharkfinMQAdminclient)
#delete_queue(sharkfinMQAdminclient)

In [ ]:
list_queues(sharkfinMQAdminclient)

In [ ]:
get_queue_runtime_properties(sharkfinMQAdminclient, containerID)

In [ ]:
sharkfinMQMessageClient = ServiceBusClient.from_connection_string(CONNECTION_STR)
client.get_queue_sender(queue_name) as sender:
        # Sending a single message
        single_message = ServiceBusMessage("Single message")
        sender.send_messages(single_message)

        # Sending a list of messages
        messages = [ServiceBusMessage("First message"), ServiceBusMessage("Second message")]
        sender.send_messages(messages)

In [ ]:
ServiceBusMessage.__dict__